In [58]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp

In [59]:
# Capacity and Unavailability 
df = pd.read_csv('./data/Cgens_Ugens.csv')
# Capacity of generators
Cgens = df['Cgens'].values.tolist()
# Probability of failure of generators (unavailability)
Ugens = df['Ugens'].values.tolist()
# Wind DF
df = pd.read_csv('./data/load_Pwind.csv')
# Hourly load
NLh_load = df['NLh_load'].values.tolist()
# Power of first offshore wind scenario
P_off_wind1 = df['P_off_wind1'].values.tolist()
# Power of second offshore wind scenario
P_off_wind2 = df['P_off_wind2'].values.tolist()
# Power of third offshore wind scenario
P_off_wind3 = df['P_off_wind3'].values.tolist()
# Round to a multiple of this (50)
P_round = 50

## Question 4
+ What is the size of the smallest and largest generator?
+ What are the smallest and largest unavailabilities?
+ What is the total installed capacity of the generation system?
+ What is the peak load?

In [60]:
# Smallest generator
smallest_generator = np.min(Cgens)
print('Smallest generator: ', smallest_generator)
# Largest generator
largest_generator = np.max(Cgens)
print('Largest generator: ', largest_generator)
# Total capacity
Ctot = np.sum(Cgens)
print('Total capacity: ', Ctot)
# Smallest unavailability
smallest_unavailability = np.min(Ugens)
print('Smallest unavailability: ', smallest_unavailability)
# Largest unavailability
largest_unavailability = np.max(Ugens)
print('Largest unavailability: ', largest_unavailability)

Smallest generator:  101.0
Largest generator:  800.0
Total capacity:  24994.0
Smallest unavailability:  0.0614
Largest unavailability:  0.0899


In [61]:

def COPT_function(Cgens,Ugens,P_round):
    # Eduardo Jerez, based on Bart Tuinema's Matlab code
    # COPT calculation
    # COPT = COPT_function(Cgens,Ugens,P_round)
    # Cgens = unit capacities
    # Ugens = Generator unavailabilities (FOR's)
    # P_round = capacities are rounded to this value

    Cgens = np.multiply(P_round,np.round(np.divide(Cgens,P_round)))  # round capacities
    P_total = sum(Cgens)  # total capacity
    n_units = len(Cgens)  # number of units

    COPT = np.zeros([np.int_(P_total/P_round), 2])
    COPT[:,0] = range(0,np.int_(P_total),P_round)
    COPT[0,1] = 1
    for i in range(n_units):
        COPT2 = np.multiply(COPT[:,1],Ugens[i])  # COPT when unit i is off
        COPT[:,1] = np.multiply(COPT[:,1],(1-Ugens[i]))  # COPT when unit i is on
        COPT[:,1] = np.add(COPT[:,1], np.hstack([np.zeros([np.int_(Cgens[i]/P_round)]), COPT2[0:-np.int_((Cgens[i]/P_round))]]))    #COPT2 is shifted down by the capacity of the generator

    #COPT = [COPT zeros(size(COPT[0]),1)]    # adding 3rd column
    #COPT(1,3) = 1
    COPT = np.transpose(np.vstack([np.transpose(COPT), np.subtract(1,np.hstack([[0],np.cumsum(COPT[0:-1,1])]))]))
    # Convert to DataFrame and round probabilities to 4 decimals
    COPT = pd.DataFrame(COPT, columns=['Capacity', 'Probability', 'Cumulative Probability'])
    COPT['Probability'] = COPT['Probability'].round(4)
    COPT['Cumulative Probability'] = COPT['Cumulative Probability'].round(4)
    return COPT

In [62]:
# Plot probability on left y-axis and cumulative probability on right y-axis vs capacity
# Layout is 800x800, Title font 30, Axis font 20, Legend font 20, Legend at the top, Legend orientation horizontal, lines+markers
fig = sp.make_subplots(specs=[[{"secondary_y": True}]])
df_plot = COPT_function(Cgens,Ugens,P_round)
fig.add_trace(go.Scatter(x=df_plot['Capacity'], y=df_plot['Probability'], name='Probability', line=dict(color='firebrick', width=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=df_plot['Capacity'], y=df_plot['Cumulative Probability'], name='Cumulative Probability', line=dict(color='royalblue', width=4)), secondary_y=True)
fig.update_layout(title='Probability and Cumulative Probability vs Capacity', 
                  title_font_size=30, 
                  xaxis_title='Capacity (MW)', 
                  yaxis_title='Probability', 
                  yaxis2_title='Cumulative Probability', 
                  font_size=20, 
                  legend_font_size=20, 
                width=800, height=800)
# Update legend position
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

